<a href="https://colab.research.google.com/github/shub-codes/HateSpeechDetection/blob/main/Hate_speech_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# libraries to be imported
1.   pandas - analyse, clean,explore and manipulate the dataset .<br>
2.   re or regex - A RegEx, or Regular Expression, is a sequence of characters that forms a search pattern. RegEx can be used to check if a string contains the specified search pattern.<br>
3. nltk- natural language transformer kit.<br>


In [10]:
import pandas as pd #to read and write csv files
import numpy as np
import nltk # natural language toolkit
import re #regular expressions
from nltk.corpus import stopwords # to remove words like a, the,is etc. such words contribute almost nothing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras import backend as K
